<a href="https://colab.research.google.com/github/RogerHeederer/NLP_practice_BERT_O/blob/master/Sentiment_BOW_tok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

본 학습의 레퍼런스는 KEPCO 자연어 처리 교육 / 정유채 교수님 자료이며, 개별 학습을 위한 용도임을 밝힙니다.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


##BOW 이용해서 긍정 부정 분류하기

In [2]:
import codecs
import pandas as pd

In [3]:
def read_data(filename):
    with codecs.open(filename, 'r', 'utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data


In [4]:
train_data = read_data('/content/gdrive/My Drive/RogerHeederer/NLP_practice_BERT_O/nsm/small_ratings_train_tok.txt')
test_data = read_data('/content/gdrive/My Drive/RogerHeederer/NLP_practice_BERT_O/nsm/small_ratings_test_tok.txt')

In [5]:
pd.DataFrame(train_data)

,0,1,2
0,9976970,아 더빙 .. 진짜 짜증나네요 목소리,0
1,3819312,흠 ... 포스터 보고 초딩 영화 줄 .... 오버 연기 조차 가볍지 않구나,1
2,10265843,너 무재 밓었 다그 래서 보는것을 추천 한 다,0
3,9045019,교도소 이야기 구먼 .. 솔직히 재미 는 없다 .. 평점 조정,0
4,6483659,사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 ! 스파이더맨 에서 늙어 보이기만...,1
...,...,...,...
9995,8665166,곰 티비 로 무료 로 봤기 때문 에 5 점주 려고 했는데 1 한국 공포영화 의 특징...,0
9996,8312675,이딴 걸 드라마 라고 썼냐 수습 할수 없으면 걍 친 자녀 아니면되 고 간단하네 얼굴...,0
9997,6386483,왠지 김연아 크면 에리 카 처럼 될것 같음 .,1
9998,4452600,솔직히 굿 ㅋㅋㅋㅋ 넘버 11 씨 는 살아남길 바랬는데 2 번 째 극장판 어서 나오길,1


In [6]:
pd.DataFrame(test_data)

,0,1,2
0,7898805,"음악 이 주가 된 , 최고 의 음악 영화",1
1,7462111,괜찮네요 오랜 만 포켓몬스터 잼밌 어 요,1
2,6900881,청춘 은 아름답다 그 아름다 움 은 이성 을 흔들어 놓는다 . 찰나 의 아름다 움 ...,1
3,9629375,눈 에 보이는 반전 이었지만 영화 의 흡인 력 은 사라지지 않았다 .,1
4,2968565,최고 !!!!!!!!!!!!!!!!,1
...,...,...,...
95,142684,역시 영화 를 볼 때 도 도덕성 을 버려 둘 수 는 없네요,0
96,9644499,이 게 뭐 가 다 들 재밌다는거지 ㅋㅋㅋ 마지막 반전 이 가장 최악 이었는데 ...,0
97,1678402,"두 주인공 의 관계 와 로맨스 는 어색하고 , 요술 인지 마법 인지 .. 뭘 하는 건지",0
98,366896,왜 헤라클레스 의 인물 묘사 를 저따위 로 했을까 ?,0


In [7]:
#훈련 데이터
train_docs_X = [col[1] for col in train_data] # 첫번째 열 값들 빼오기(리뷰 텍스트)
train_Y = [col[2] for col in train_data] # 두번째 열 값들 빼오기 (긍정,부정 레이블)

In [8]:
train_docs_X[1], train_Y[1] #리뷰 텍스트 및 평가

('흠 ... 포스터 보고 초딩 영화 줄 .... 오버 연기 조차 가볍지 않구나', '1')

In [9]:
#테스트 데이터
test_docs_X = [col[1] for col in test_data] # 첫번째 열 값들 빼오기(리뷰 텍스트)
test_Y = [col[2] for col in test_data] # 두번째 열 값들 빼오기 (긍정,부정 레이블)

In [10]:
test_docs_X[1], test_Y[1]

('괜찮네요 오랜 만 포켓몬스터 잼밌 어 요', '1')

#### CountVectorizer 변환
데이터 : ['I am happy', 'I am sad']<br>
단어 모음 : ['I', 'am', 'happy', 'sad']<br>
변환 결과 : [[1, 1, 1, 0], [1, 1, 0, 1]]<br>

## 단어 갯수 카운팅해서 고정된 크기의 벡터를 만들어줌

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(max_features = 1000).fit(train_docs_X)

In [12]:
vec

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=1000, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [13]:
train_X = vec.transform(train_docs_X).toarray()
test_X = vec.transform(test_docs_X).toarray()

In [14]:
print(type(train_docs_X))
len(train_docs_X)

<class 'list'>


10000

In [15]:
print(type(train_X))
train_X.shape

<class 'numpy.ndarray'>


(10000, 1000)

In [16]:
print(type(train_Y))
len(train_Y)

<class 'list'>


10000

In [17]:
train_X[1:10]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [18]:
train_Y[1:10]

['1', '0', '0', '1', '0', '0', '0', '1', '1']

In [19]:
%%time
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb.fit(train_X, train_Y)

CPU times: user 173 ms, sys: 18.4 ms, total: 192 ms
Wall time: 222 ms


In [20]:
print("Accuracy on training set: {:.3f}".format(gnb.score(train_X, train_Y)))
print("Accuracy on test set: {:.3f}".format(gnb.score(test_X, test_Y)))

Accuracy on training set: 0.749
Accuracy on test set: 0.720


In [21]:
pos = [ [test_docs_X[i], test_Y[i], gnb.predict([test_X[i]]) ] for i in range(len(test_Y)) if test_Y[i] == '1' ]
neg = [ [test_docs_X[i], test_Y[i], gnb.predict([test_X[i]]) ] for i in range(len(test_Y)) if test_Y[i] == '0' ]

In [22]:
print('긍정 리뷰 예측')
for i in pos[0:5]:
  print('입력 : ', i[0])
  print('정답 : ', i[1])
  print('출력 : ', i[2])

긍정 리뷰 예측
입력 :  음악 이 주가 된 , 최고 의 음악 영화
정답 :  1
출력 :  ['1']
입력 :  괜찮네요 오랜 만 포켓몬스터 잼밌 어 요
정답 :  1
출력 :  ['1']
입력 :  청춘 은 아름답다 그 아름다 움 은 이성 을 흔들어 놓는다 . 찰나 의 아름다 움 을 잘 포착 한 섬세하고 아름다운 수채화 같은 퀴어 영화 이다 .
정답 :  1
출력 :  ['1']
입력 :  눈 에 보이는 반전 이었지만 영화 의 흡인 력 은 사라지지 않았다 .
정답 :  1
출력 :  ['0']
입력 :  최고 !!!!!!!!!!!!!!!!
정답 :  1
출력 :  ['1']


In [23]:
print ('\n부정 리뷰 예측')
for i in neg[0:5]:
    print ('입력', i[0])
    print ('정답', i[1])
    print ('출력', i[2])


부정 리뷰 예측
입력 뭐 야 이 평점 들 은 .... 나쁘진 않지만 10 점 짜 리 는 더 더욱 아니잖아
정답 0
출력 ['1']
입력 지루하지는 않은데 완전 막장 임 ... 돈 주고 보기 에는 ....
정답 0
출력 ['0']
입력 3 D 만 아니었어도 별 다섯 개 줬을텐데 .. 왜 3 D 로 나와서 제 심기 를 불편하게 하죠 ??
정답 0
출력 ['0']
입력 진정한 쓰레기
정답 0
출력 ['1']
입력 한국 독립영화 의 한계 그렇게 아버지 가 된다와 비교 됨
정답 0
출력 ['1']


##KNN 모델 이용

In [24]:
%%time
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=9)
neigh.fit(train_X, train_Y)

CPU times: user 1.31 s, sys: 4.12 ms, total: 1.31 s
Wall time: 1.49 s


In [25]:
print("Accuracy on training set: {:.3f}".format(neigh.score(train_X, train_Y)))
print("Accuracy on test set: {:.3f}".format(neigh.score(test_X, test_Y)))

Accuracy on training set: 0.722
Accuracy on test set: 0.690


In [26]:
pos = [ [test_docs_X[i], test_Y[i], neigh.predict([test_X[i]]) ] for i in range(len(test_Y)) if test_Y[i] == '1' ]
neg = [ [test_docs_X[i], test_Y[i], neigh.predict([test_X[i]]) ] for i in range(len(test_Y)) if test_Y[i] == '0' ]

print ('Predicted as Positive:')
for i in pos[:5]:
    print ('입력', i[0])
    print ('정답', i[1])
    print ('출력', i[2])
print ('\nPredicted as Negative:')
for i in neg[:5]:
    print ('입력', i[0])
    print ('정답', i[1])
    print ('출력', i[2])


Predicted as Positive:
입력 음악 이 주가 된 , 최고 의 음악 영화
정답 1
출력 ['1']
입력 괜찮네요 오랜 만 포켓몬스터 잼밌 어 요
정답 1
출력 ['0']
입력 청춘 은 아름답다 그 아름다 움 은 이성 을 흔들어 놓는다 . 찰나 의 아름다 움 을 잘 포착 한 섬세하고 아름다운 수채화 같은 퀴어 영화 이다 .
정답 1
출력 ['0']
입력 눈 에 보이는 반전 이었지만 영화 의 흡인 력 은 사라지지 않았다 .
정답 1
출력 ['0']
입력 최고 !!!!!!!!!!!!!!!!
정답 1
출력 ['1']

Predicted as Negative:
입력 뭐 야 이 평점 들 은 .... 나쁘진 않지만 10 점 짜 리 는 더 더욱 아니잖아
정답 0
출력 ['1']
입력 지루하지는 않은데 완전 막장 임 ... 돈 주고 보기 에는 ....
정답 0
출력 ['1']
입력 3 D 만 아니었어도 별 다섯 개 줬을텐데 .. 왜 3 D 로 나와서 제 심기 를 불편하게 하죠 ??
정답 0
출력 ['0']
입력 진정한 쓰레기
정답 0
출력 ['0']
입력 한국 독립영화 의 한계 그렇게 아버지 가 된다와 비교 됨
정답 0
출력 ['0']
